In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from datacollector.datacollector import TennisDataCollector
from dataprocessor.dataprocessor import TennisDataProcessor

In [3]:
collector = TennisDataCollector()
data = collector.get_all_data(max_tournaments=1)

HTTPError: 403 Client Error: Forbidden for url: https://www.sofascore.com/api/v1/config/default-unique-tournaments/NL/tennis

In [188]:
processor = TennisDataProcessor()
df = processor.process_all_data()

INFO:root:Processing cuptree file: cuptrees_2363_11172.json
INFO:root:Processing cuptree file: cuptrees_2363_11182.json
INFO:root:Processing cuptree file: cuptrees_2363_11183.json
INFO:root:Processing cuptree file: cuptrees_2363_11187.json
INFO:root:Processing cuptree file: cuptrees_2363_11189.json
INFO:root:Processing cuptree file: cuptrees_2363_11192.json
INFO:root:Processing cuptree file: cuptrees_2363_11195.json
INFO:root:Processing cuptree file: cuptrees_2363_11196.json
INFO:root:Processing cuptree file: cuptrees_2363_12805.json
INFO:root:Processing cuptree file: cuptrees_2363_15801.json
INFO:root:Processing cuptree file: cuptrees_2363_20084.json
INFO:root:Processing cuptree file: cuptrees_2363_26329.json
INFO:root:Processing cuptree file: cuptrees_2363_35210.json
INFO:root:Processing cuptree file: cuptrees_2363_40339.json
INFO:root:Processing cuptree file: cuptrees_2363_48186.json
INFO:root:Processing cuptree file: cuptrees_2363_55443.json
INFO:root:Processing cuptree file: cuptr

In [192]:
from trainset.trainset import TennisTrainset

In [198]:
trainset = TennisTrainset()

Participants or games data file not found. Please ensure the files exist in the specified path.


In [199]:
trainset.participants

In [189]:
duplicates = df['participants']['name'].duplicated(keep=False)
if duplicates.any():
    print("Duplicate names found:")
    print(df['participants'][duplicates]['name'].value_counts())
else:
    print("No duplicate names found.")

No duplicate names found.


In [190]:
df['participants'].sort_values(by='ranking')

,name,slug,shortName,gender,nameCode,ranking,disabled,national,id
380,Jannik Sinner,sinner-jannik,J. Sinner,M,SIN,1.0,False,False,206570
402,Carlos Alcaraz,alcaraz-carlos,C. Alcaraz,M,ALC,2.0,False,False,275923
191,Alexander Zverev,zverev-alexander,A. Zverev,M,ZVE,3.0,NaN,False,57163
507,Jack Draper,draper-jack,J. Draper,M,DRA,4.0,False,False,258749
185,Taylor Fritz,fritz-taylor,T. Fritz,M,FRI,5.0,False,False,136042
...,...,...,...,...,...,...,...,...,...
482,Filippo Baldi,baldi-filippo,F. Baldi,M,BAL,NaN,NaN,False,84253
488,Andrea Arnaboldi,arnaboldi-andrea,A. Arnaboldi,M,AAR,NaN,NaN,False,16057
490,Jason Jung,jung-jason,J. Jung,M,JJU,NaN,NaN,False,63446
491,Tim van Rijthoven,van-rijthoven-tim,T. van Rijthoven,M,VAN,NaN,False,False,102411


In [180]:
# Show info and first few rows
df_games = df['games']
df_games.info()
display(df_games.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2719 entries, 0 to 2718
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   finished           2719 non-null   bool          
 1   result             2719 non-null   object        
 2   homeTeamScore      2664 non-null   object        
 3   awayTeamScore      2664 non-null   object        
 4   hasNextRoundLink   2668 non-null   object        
 5   id                 2719 non-null   int64         
 6   events             2719 non-null   object        
 7   seriesStartDate    2719 non-null   datetime64[ns]
 8   month              2719 non-null   int32         
 9   home_id            2719 non-null   int64         
 10  away_id            2719 non-null   int64         
 11  round_description  2719 non-null   object        
dtypes: bool(1), datetime64[ns](1), int32(1), int64(3), object(6)
memory usage: 225.8+ KB


,finished,result,homeTeamScore,awayTeamScore,hasNextRoundLink,id,events,seriesStartDate,month,home_id,away_id,round_description
0,True,3:0,3,0,True,388579,[6654894],2015-01-20 03:35:00,1,702241,702240,R128
1,True,3:1,3,1,True,388589,[6653593],2015-01-20 06:15:00,1,702260,702261,R128
2,True,3:2,3,2,True,388635,[6654901],2015-01-20 04:30:00,1,702353,702352,R128
3,True,1:3,1,3,True,388609,[6653598],2015-01-20 01:35:00,1,702301,702300,R128
4,True,3:0,3,0,True,388641,[6654893],2015-01-20 01:55:00,1,702365,702364,R128


In [181]:
# Symmetrize matches: add a row for each match with home/away swapped and result reversed
def reverse_result(result):
    # Handles results like '3:1', '2:3', etc.
    if isinstance(result, str) and ':' in result:
        parts = result.split(':')
        return f"{parts[1]}:{parts[0]}"
    return result

df_games_sym = df_games.copy()
swapped = df_games.copy()

# Swap home and away columns
swapped['home_id'], swapped['away_id'] = df_games['away_id'], df_games['home_id']
swapped['homeTeamScore'], swapped['awayTeamScore'] = df_games['awayTeamScore'], df_games['homeTeamScore']
swapped['result'] = df_games['result'].apply(reverse_result)

# Optionally, you may want to swap other columns if needed (e.g., events, round_description)
# For now, keep other columns as is

# Concatenate original and swapped
df_games_sym = pd.concat([df_games, swapped], ignore_index=True)
df_games_sym.reset_index(drop=True, inplace=True)

print(f"Original matches: {len(df_games)}, Symmetrized matches: {len(df_games_sym)}")
display(df_games_sym.head())

Original matches: 2719, Symmetrized matches: 5438


,finished,result,homeTeamScore,awayTeamScore,hasNextRoundLink,id,events,seriesStartDate,month,home_id,away_id,round_description
0,True,3:0,3,0,True,388579,[6654894],2015-01-20 03:35:00,1,702241,702240,R128
1,True,3:1,3,1,True,388589,[6653593],2015-01-20 06:15:00,1,702260,702261,R128
2,True,3:2,3,2,True,388635,[6654901],2015-01-20 04:30:00,1,702353,702352,R128
3,True,1:3,1,3,True,388609,[6653598],2015-01-20 01:35:00,1,702301,702300,R128
4,True,3:0,3,0,True,388641,[6654893],2015-01-20 01:55:00,1,702365,702364,R128


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

df_games_clean = df_games.dropna(subset=['homeTeamScore', 'awayTeamScore', 'result'])
X = df_games_clean[['home_id', 'away_id']].astype(int)
y = df_games_clean['result'].apply(lambda x: 1 if x.startswith('3:') else 0)  # 1: home wins, 0: otherwise

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# Predict and evaluate
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))